In [94]:

import nbimporter

from Database_connection.database_connection import connect_db
import requests
import pandas as pd
import numpy as np
from datetime import datetime



## Protocols General Data


In [80]:
response = requests.get('https://api.llama.fi/protocols')
data = response.json()
df = pd.DataFrame.from_dict(data, orient = 'Columns')

In [82]:
df = df [[
       'id', 'name', 'address', 'symbol', 'url', 'description', 'chain',
       'logo', 'audits', 'gecko_id', 'category',
#        'chains', 
    'module', 'twitter', 'listedAt',
       'slug', 'tvl',
#     'chainTvls', 'forkedFrom', 'oracles', 
    'change_1h', 'change_1d', 'change_7d',
       # 'tokenBreakdowns', 
]]

In [75]:
# df = df[['id', 'name', 'address', 'symbol', 'url', 'description', 'chain',
#        'logo', 'audits', 'gecko_id', 'cmcId', 'category',
#        'module', 'twitter', 'listedAt',
#        'slug', 'tvl', 
#        # 'chainTvls','chains','tokenBreakdowns', 'forkedFrom', 'oracles' Creates troble cuase its a dict 
#        'change_1h', 'change_1d', 'change_7d',
#        'mcap', 'treasury', 'audit_links',
#        'governanceID', 'github', 'parentProtocol',
#        'staking', 'pool2']]

KeyError: "['cmcId', 'mcap', 'treasury', 'audit_links', 'governanceID', 'github', 'parentProtocol', 'staking', 'pool2'] not in index"

In [84]:
df.isna().sum().sort_values(ascending = False)

# df.columns

gecko_id       2051
address        1119
change_7d       724
listedAt        637
change_1d       624
change_1h       616
twitter          64
audits           10
description       1
symbol            0
url               0
chain             0
logo              0
name              0
category          0
module            0
slug              0
tvl               0
id                0
dtype: int64

In [83]:

with connect_db('protocols_tvl') as connection:
    df.to_sql('general', con=connection, if_exists='replace', index=False)
    print(f'All data inserted into general')


## Protocols. Actual TVL by Chain

In [85]:
df = pd.DataFrame.from_dict(data, orient = 'Columns')

In [92]:
with connect_db('protocols_tvl') as connection:
    # Create an empty list to hold all the data
    all_data = []

    # Iterate through rows in the DataFrame
    for index, row in df.iterrows():
        name = row['slug']
        chains_data = row['chainTvls']

        # Convert the dictionary of values into a list of dictionaries with 'chain' and 'value' columns
        chains_data_list = [{'chain': chain, 'value': value} for chain, value in chains_data.items()]

        # Add the 'id' column to each dictionary
        for data in chains_data_list:
            data['id'] = name

        # Extend the 'all_data' list with the current data
        all_data.extend(chains_data_list)
        # print(f'{name}')

    # Create a DataFrame from the combined data
    superbig_df = pd.DataFrame(all_data)
    timestamp = datetime.now()
    superbig_df['updated_at'] = timestamp

    # Insert the combined data into the database as a single table
    table_name = "by_chain"  # Choose a suitable table name
    superbig_df.to_sql(table_name, con=connection, if_exists='replace')

    print(f'All data inserted into {table_name}')

All data inserted into by_chain
